In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import PIL
import math
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from random import randint
from utils import get_stats, get_loaders
from dataset import NormalizedDataset, IncompleteDataset

In [2]:
white = 255
black = 0

def inverse_color(img):
    return PIL.Image.eval(img, lambda val: 255 - val)

def fix_background_color_bug(img):
    colors = sorted(img.getcolors(), key=lambda pair: pair[0], reverse=True)
    replace_color = colors[0][1]
    remove_color = colors[2][1] if colors[2][1] < colors[1][1] else colors[1][1]

    data = np.array(img)
    data[data == remove_color] = replace_color
    return PIL.Image.fromarray(data)

def remove_background(img):
    return PIL.Image.eval(img, lambda val: 0 if val < (256/2) else val)

def to_binary(img):
    return PIL.Image.eval(img, lambda val: 255 if val < (256/2) else 0)

In [8]:
binary_image = True

## Rotation, Brightness and Resizing
random_transforms = [
    transforms.RandomRotation(360, fill=black),
    transforms.RandomAffine(0, shear=15, scale=(0.8, 0.95), translate=(.03, .03)),
]

format_transforms = []

if not binary_image:
    random_transforms.append(transforms.ColorJitter(brightness=(0.9, 1), contrast=(0.7, 1)))
else:
    format_transforms.append(transforms.Lambda(to_binary))

all_transforms = transforms.Compose(random_transforms + format_transforms)

video_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Lambda(inverse_color),
    transforms.Lambda(remove_background),
    transforms.Resize((28, 28)),
    transforms.Compose(format_transforms),
])

minst_tensor_transform = transforms.Compose([
    all_transforms,
    transforms.ToTensor(),
])

video_tensor_transform = transforms.Compose([
    video_transform,
    transforms.ToTensor(),
])

In [9]:
minst_video_dataset = datasets.ImageFolder(root='video_dataset/minst', transform=video_tensor_transform)


datasets.ImageFolder(root='video_dataset/minst', transform=video_transform)[2][0]

In [10]:
# Minst Dataset
minst_dataset = NormalizedDataset(datasets.MNIST("", transform=minst_tensor_transform, download=True))

# Show example image
datasets.MNIST("", transform=all_transforms, download=True)[0][0]

In [6]:
# Creating a dataset and loader with only the video images
minst_mean, minst_std = get_stats(minst_dataset)
minst_classes = [word[:1] for word in minst_dataset.classes]
minst_incomplete_dataset = IncompleteDataset(minst_video_dataset, minst_classes, minst_mean, minst_std)
video_test_loader = torch.utils.data.DataLoader(minst_incomplete_dataset, batch_size=100)

In [7]:
# Creating train and test loader from MINST
train_loader, test_loader = get_loaders(minst_dataset)
len(train_loader.dataset), len(test_loader.dataset)

(60000, 60000)

In [162]:
# from keras.regularizers import L1L2
# from keras.layers import Conv2D
# Creating a Net class object, which consists of 2 convolutional layers, max-pool layers and fully-connected layers
class Conv_Net(nn.Module):
    
    def __init__(self, nb_hidden=50):        
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)  # the first convolutional layer, which processes the input image
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set

        self.fc1 = nn.Linear(64, nb_hidden)  # the first fully-connected layer, which gets flattened max-pooled set
        self.fc2 = nn.Linear(nb_hidden, 10)  # the second fully-connected layer that outputs the result

    # Creating the forward pass
    def forward(self, x):
        
        # The first two layers
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
                
        # The second two layers
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) 
                
        x = F.relu(F.max_pool2d(self.conv3(x), kernel_size=2)) 

        # Flattening the data set for fully-connected layer
        x = x.view(x.size(0), -1)
    
        # The first fully-connected layer
        x = F.relu(self.fc1(x))
        
        x = nn.Dropout(p=0.3)(x)
        
        # The second full-connected layer
        x = self.fc2(x)
        
        return x

In [164]:
%%time

model = Conv_Net()

losses = []
test_losses = []

# Defining the optimizer for GD
lr = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr = lr) 

# Defining the criterion to calculate loss
criterion = nn.CrossEntropyLoss()

# Train the model
nb_epochs = 300
best_acc = 0


for e in range(nb_epochs):
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    for train_input, train_target in train_loader:

        # Model computations
        output = model(train_input)
        loss = criterion(output, train_target) 
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
        tmp_acc = 0
        for tensor, target in zip(output, train_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
                
        tmp_acc /= len(output)
        acc += tmp_acc 
        
    acc /= len(train_loader)
        
    should_print = e%1 == 0
    if should_print:
        print('%dth epoch, train acc: %f' % (e+1, acc), end="")
    losses.append(loss)
    
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    for test_input, test_target in test_loader:
        
        output = model(test_input)
        loss = criterion(output, test_target) 
        
        tmp_acc = 0
        for tensor, target in zip(output, test_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
            elif should_print:
                print(", {} != {}".format(index, target), end="")
                
        tmp_acc /= len(output)
        acc += tmp_acc 
                
    acc /= len(test_loader)
    print(", test acc: %f" % (acc))
    
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "digit_model")
    
    test_losses.append(loss)

1th epoch, train acc: 0.367333, 2 != 3, 8 != 3, 2 != 7, 0 != 7, test acc: 0.333333
2th epoch, train acc: 0.540583, 8 != 2, 0 != 7, 0 != 7, test acc: 0.500000
3th epoch, train acc: 0.610700, 0 != 7, 0 != 7, test acc: 0.666667
4th epoch, train acc: 0.656950, 0 != 7, 0 != 7, test acc: 0.666667
5th epoch, train acc: 0.690017, 2 != 3, 0 != 7, 0 != 7, test acc: 0.500000
6th epoch, train acc: 0.715817, 0 != 7, 3 != 7, test acc: 0.666667
7th epoch, train acc: 0.735133, 2 != 7, 0 != 7, test acc: 0.666667
8th epoch, train acc: 0.751283, 7 != 2, 0 != 7, 0 != 7, test acc: 0.500000
9th epoch, train acc: 0.765883, 7 != 2, 2 != 3, 2 != 7, 5 != 7, test acc: 0.333333
10th epoch, train acc: 0.772967, 0 != 7, 0 != 7, test acc: 0.666667
11th epoch, train acc: 0.783050, 7 != 2, 0 != 7, 0 != 7, test acc: 0.500000
12th epoch, train acc: 0.788083, 3 != 7, 3 != 7, test acc: 0.666667
13th epoch, train acc: 0.797667, 2 != 3, 0 != 7, 0 != 7, test acc: 0.500000
14th epoch, train acc: 0.799867, 0 != 7, 3 != 7, test

KeyboardInterrupt: 

In [94]:
model = Conv_Net()
model.load_state_dict(torch.load("digit_model"))
criterion = nn.CrossEntropyLoss()

RuntimeError: Error(s) in loading state_dict for Conv_Net:
	size mismatch for fc1.weight: copying a param with shape torch.Size([50, 576]) from checkpoint, the shape in current model is torch.Size([40, 64]).
	size mismatch for fc1.bias: copying a param with shape torch.Size([50]) from checkpoint, the shape in current model is torch.Size([40]).
	size mismatch for fc2.weight: copying a param with shape torch.Size([15, 50]) from checkpoint, the shape in current model is torch.Size([10, 40]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([15]) from checkpoint, the shape in current model is torch.Size([10]).

In [136]:
loader = torch.utils.data.DataLoader(minst_incomplete_dataset, batch_size=10)

In [137]:
acc = 0
for test_input, test_target in loader:

    print(test_input.shape)
    output = model(test_input)
    loss = criterion(output, test_target) 

    tmp_acc = 0
    for tensor, target in zip(output, test_target):
#         print(tensor, target)
        _, index = tensor.max(0)
#         print(tensor, index)
        if index == target:
            tmp_acc += 1

    tmp_acc /= len(output)
    acc += tmp_acc 

acc /= len(test_loader)
print("test loss: %f, acc: %f" % (loss, acc))

torch.Size([6, 1, 30, 30])


RuntimeError: size mismatch, m1: [6 x 256], m2: [64 x 40] at ../aten/src/TH/generic/THTensorMath.cpp:41